# Classificação de Imagens de 03 classes de plantas por tipos de doenças, com Transfer Learning

#### Objetivo: Realizar o Teste de Inferência com imagens de testes de classe, para o cenário do Caso de Teste 02.

##### Classes Consideradas:
###### Grape___Black_rot
###### Grape___Esca_(Black_Measles)
###### Grape___healthy


## Consultas:
##### https://keras.io/applications/
##### https://www.tensorflow.org/tutorials/load_data/images
##### https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
##### https://keras.io/api/data_loading/
##### https://datascience.stackexchange.com/questions/81541/how-to-predict-multiple-images-from-folder-in-python

## Leitura do local da base de dados

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
workdir_path = '/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive' #MODIFICAR!!! #Caminho para o seu workspace
os.chdir(workdir_path)

Mounted at /content/drive


In [2]:
os.path.abspath(os.getcwd())

'/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive'

In [3]:
# Base de imagens para treino com total de 4659 imagens (classes de tipos de platas)
trainpath = workdir_path + '/classeuva/Train'

# Base de imagens para validação com total 1169 imagens (classes de tipos de platas)
validpath = workdir_path + '/classeuva/Valid'

# Base de imagens para validação com total de 30 imagens (classes de tipos de platas)
testpath = workdir_path + '/classeuva/Test'


## Leitura de Bibliotecas

In [4]:
from tensorflow.keras.models import load_model
import argparse
import pickle
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import numpy as np
import logging, os
import sys 
from keras.preprocessing import image
import tensorflow as tf
import math
import operator
from pathlib import Path


## Leitura do modelo do respectivo cenário 06 para o Caso de Teste 02

In [5]:
# Ler modelo treinado gravado para submeter aos dados de testes
modelosalvo = tf.keras.models.load_model('/content/drive/MyDrive/TrabalhoFinal/DadosTCC/archive/Inferência-TestesModelos/Caso de Teste 02/Resnet50-cenário06/Modelo_Salvo_1/Resnet50.h5')

## Carrega as Imagens de Testes Selecionadas

In [6]:
# Prepara variáveis para imagens do tipo Healphy

healthy_path = testpath +'/healthy/' # Caminho das imagens

imagens_healthy = [] # Inicializa uma lista para carregar as imagens


In [7]:
# Carrega as imagens para do tipo Healphy
for img in os.listdir(healthy_path):                                 # Cria um loop
        img = os.path.join(healthy_path, img)                        
        img = tf.keras.utils.load_img(img, target_size=(224,224))    # coloca a imagem no shape 
        img = tf.keras.utils.img_to_array(img)                       # transforma em um array
        img = np.expand_dims(img, axis=0)                            
        imagem_normalziada = np.array(img, dtype="float") / 255.0    # normaliza a imagem dividindo por 255
        imagens_healthy.append(imagem_normalziada)                   # carrega a imagem para a lista 
        
imagens_healthy = np.vstack(imagens_healthy)                         

## Previsão Dados de Teste (classe Healphy)

In [8]:
# faz a previsao das imagens de teste do tipo Healphy
dados_previsao_imagens_healthy = []                                 # inicializa uma lista para colocar o reultado da previsao das imagens na lista
nome_arquivos = []                                                  # inicializa uma lista para colocar o nome dos arquivos
for arquivo in os.listdir(healthy_path):                            # inicializa um loop (para cada arquivo dentro da pasta )
    pred_result = modelosalvo.predict(imagens_healthy)              # faz a previsao de cada arquivo
    dados_previsao_imagens_healthy.append(pred_result)              # coloca os dados da previsão na lsita 
    nome_arquivos.append(arquivo)                                   # coloca o nome do arquivo na lista "nome_arquivos"

1/1 [==============================] - 0s 30ms/step


In [9]:
dados_previsao_imagens_healthy[0]

array([[0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [8.6343418e-05, 6.6740620e-07, 9.9991298e-01],
       [7.9882600e-14, 5.4435952e-21, 1.0000000e+00],
       [2.8709034e-37, 0.0000000e+00, 1.0000000e+00],
       [1.1256288e-33, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [7.4796302e-09, 9.8038939e-15, 1.0000000e+00],
       [2.1325047e-38, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]], dtype=float32)

In [11]:
# Define os labels R:Grape___Black_rot, M:Grape___Esca_(Black_Measles) e H:Grape___healthy' na mesma ordem de classes do modelo na etapa de load.
# Previsão dados de testes selecionados

label_file_path = os.path.join("RMH", 'labels') 

#SORT
top_k = []
pred = [] 
for i in range(len(dados_previsao_imagens_healthy)):
            ordem = dados_previsao_imagens_healthy[i][0].argsort()[-len(dados_previsao_imagens_healthy[i][0]):][::-1]
            top_k.append(ordem)
            top = top_k[i][:10]
            print(nome_arquivos[i])
            for node_id in top:
                planta_string = label_file_path[node_id]
                score = dados_previsao_imagens_healthy[i][0][node_id]
                print('%s (score = %.5f)' % (planta_string, score))

29ffbe00-4f75-496b-8bb5-dc1be1eb73b3___Mt.N.V_HL 8956_90deg.JPG
H (score = 1.00000)
M (score = 0.00000)
R (score = 0.00000)
02f95acb-5d92-4f2a-b7ec-3af8709ee7c9___Mt.N.V_HL 9078_270deg.JPG
H (score = 1.00000)
M (score = 0.00000)
R (score = 0.00000)
2bd2ff11-36dc-43ec-888c-efa38601ef88___Mt.N.V_HL 8907_270deg.JPG
H (score = 1.00000)
M (score = 0.00000)
R (score = 0.00000)
1c13c447-4d2d-4eac-9835-705d952b09d6___Mt.N.V_HL 9128_270deg.JPG
H (score = 1.00000)
M (score = 0.00000)
R (score = 0.00000)
1180dbb8-4509-4f56-a5f6-15ceac163ff9___Mt.N.V_HL 8928_90deg.JPG
H (score = 1.00000)
M (score = 0.00000)
R (score = 0.00000)
a5f23bc2-240d-4db0-8998-43a2cf2a0752___Mt.N.V_HL 6166_new30degFlipLR.JPG
H (score = 1.00000)
M (score = 0.00000)
R (score = 0.00000)
d1d71d89-a456-4ac7-9388-9d9871691382___Mt.N.V_HL 6205_270deg.JPG
H (score = 1.00000)
M (score = 0.00000)
R (score = 0.00000)
d1d71d89-a456-4ac7-9388-9d9871691382___Mt.N.V_HL 6205_90deg.JPG
H (score = 1.00000)
M (score = 0.00000)
R (score = 0.00